## Step 1: Setting up Spotify API
1. Create a Spotify account if you don't already have one: Spotify Signup.
2. Go to the Spotify Developer Dashboard and log in.
3. Click "Create an App" to generate a Client ID and Client Secret.
4. In your app settings, add http://localhost:8888/callback as a redirect URI.

Install required spotipy library:

In [1]:
pip install spotipy

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Note: you may need to restart the kernel to use updated packages.


## Step 2: Authentication

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set your client ID and client secret
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id='your_client_id',
    client_secret='your_client_secret',
    redirect_uri='http://localhost:8888/callback',
    scope="user-library-read user-read-recently-played"
))

## Step 3: Data Collection
Option 1: Classify by Users
Fetch your liked tracks or recently played tracks:

In [ ]:
import pandas as pd

def fetch_tracks(sp, fetch_function, limit=50):
    results = []
    offset = 0
    while True:
        response = fetch_function(limit=limit, offset=offset)
        items = response['items']
        if not items:
            break
        results.extend(items)
        offset += limit
    return pd.DataFrame(results)

# Fetch liked tracks
liked_tracks = fetch_tracks(sp, sp.current_user_saved_tracks)

# Fetch recently played tracks
recent_tracks = fetch_tracks(sp, sp.current_user_recently_played)


### Get audio features for tracks

In [ ]:
def fetch_audio_features(sp, track_ids):
    features = sp.audio_features(track_ids)
    return pd.DataFrame(features)

# Example: Fetch audio features for liked tracks
liked_track_ids = liked_tracks['track'].apply(lambda x: x['id'])
audio_features = fetch_audio_features(sp, liked_track_ids)


### Combine datasets with an outcome variable

In [ ]:
# Add track names for reference
audio_features['track_name'] = liked_tracks['track'].apply(lambda x: x['name'])

# Combine with another user's data
partner_audio_features = ...  # Replace with partner's dataset
audio_features['label'] = 'your_data'
partner_audio_features['label'] = 'partner_data'
combined_data = pd.concat([audio_features, partner_audio_features])


### Option 2: Classify by Genres
1. Download and inspect Kaggle dataset:

In [ ]:
spotify_data = pd.read_csv('path_to_spotify_dataset.csv')

# Filter down to two genres
genre_1_data = spotify_data[spotify_data['genre'] == 'Genre1']
genre_2_data = spotify_data[spotify_data['genre'] == 'Genre2']
combined_data = pd.concat([genre_1_data, genre_2_data])
combined_data['label'] = combined_data['genre']


## Step 4: Data Exploration
Use descriptive statistics and visualizations to explore the data.

In [ ]:
import matplotlib.pyplot as plt

# Example: Plot danceability
combined_data['danceability'].hist(by=combined_data['label'])
plt.show()

# Example: Compare energy levels
combined_data.boxplot(column='energy', by='label')
plt.show()


Step 5: Modeling
1. Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

X = combined_data.drop(columns=['label', 'track_name', 'genre'])
y = combined_data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


2. Define models

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

# KNN
knn = KNeighborsClassifier()

# Decision Tree
dt = DecisionTreeClassifier(random_state=42)

# Bagged Tree
bagged_tree = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)

# Random Forest
random_forest = RandomForestClassifier(random_state=42)


3. Train and evaluate models

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

models = [knn, dt, bagged_tree, random_forest]
model_names = ['KNN', 'Decision Tree', 'Bagged Tree', 'Random Forest']

for model, name in zip(models, model_names):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"--- {name} ---")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


Step 6. Model Comparison Summarize the model performances in a table and create visualizations for comparison

In [ ]:
import matplotlib.pyplot as plt

accuracies = [accuracy_score(y_test, model.predict(X_test)) for model in models]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Model Comparison')
plt.show()
